# Proyecto 1 -- Regresión a Pie

In [11]:
import numpy as np
import requests as rq
import io 


data = np.load('https://drive.google.com/file/d/1Y5vA7xxk68rjhon6_SJCIqhDn2mIyBZU/view?usp=sharing')
data

OSError: [Errno 22] Invalid argument: 'https://drive.google.com/file/d/1Y5vA7xxk68rjhon6_SJCIqhDn2mIyBZU/view?usp=sharing'